In [1]:
import pickle
from util_fdp import *

/root/miniconda3/envs/myconda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-11 12:16:21,877	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
# cvg_case2 = load_pickle('case_2_federated_hete_d_cvg.pkl')
# mae_case2 = load_pickle('case_2_federated_hete_d_mae.pkl')


cvg_case2 = load_pickle('case_federated_homo_cvg.pkl')
mae_case2 = load_pickle('case_federated_homo_mae.pkl')
print(cvg_case2)
print(mae_case2)

{5000: {0.3: {0.25: {1: np.float64(0.942)}}}}
{5000: {0.3: {0.25: {1: np.float64(0.02099225519472169)}}}}


In [7]:
# print(cvg_case2)
# print(mae_case2)



In [8]:
def generate_combined_table(data_ci, data_mae, caption="Results",
                            label="tab:combined"):
    # 自动获取所有n值并按数字顺序排序
    all_n = sorted(set(list(data_ci.keys()) + list(data_mae.keys())), key=int)
    latex_code = f""
    for n in all_n:
        # 添加n标题行
        latex_code += f"\\multicolumn{{6}}{{c}}{{\\textbf{{T = {n}}}}} \\\\\n"
        latex_code += "\\midrule\n"
        
        # 遍历所有tau和r组合
        for tau in sorted(data_ci[n].keys(), key=float):
            for r_val in data_ci[n][tau].keys():
                # 生成行内容
                row = f"{tau} & {r_val if isinstance(r_val, str) else r_val} "
                
                # 遍历所有E值
                for e in data_ci[n][tau][r_val].keys():
                    ci = data_ci[n][tau][r_val].get(e, "--")
                    mae = data_mae[n][tau][r_val].get(e, "--")
                    cell = f"& {ci:.3f}({mae:.4f}) " if isinstance(ci, float) and isinstance(mae, float) else "& -- "
                    row += cell
                
                latex_code += row + "\\\\\n"
            
            # 添加tau之间的分隔线
            latex_code += "\\hline\n"
    latex_code += """\\bottomrule
\\end{tabular}
\\end{table}"""
    return latex_code

In [9]:
latex_output = generate_combined_table(cvg_case2, mae_case2)
with open("combined_table.tex", "w") as f:
    f.write(latex_output)